In [1]:
import requests
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import json
import pandas as pd

In [2]:
url = 'http://apis.data.go.kr/1741000/DisasterMsg2/getDisasterMsgList'
serviceKey = '3f%2FnmTs7kFeKIVO%2FQYey91YeNYJL5W7HtOKfhVj3cr4a7coxcZmXf6zHssAg2pa9gMjgTxE%2FfpT%2B8CfWb2NEWA%3D%3D'
pageNo = '1'
numOfRows = '1000'
_type = 'json'

In [3]:
# 해당 컬럼 작성
coloumns = ["create_date"
           , "location_id"
           , "location_name"
           , "md101_sn"
           , "msg"
           , "send_platform"]

In [4]:
# 
def return_list_from_respose(List) -> list :
    return [
        [ # 내용
            List["DisasterMsg"][1]["row"][i]["create_date"]
            , List["DisasterMsg"][1]["row"][i]["location_id"]
            , List["DisasterMsg"][1]["row"][i]["location_name"]
            , List["DisasterMsg"][1]["row"][i]["md101_sn"]
            , List["DisasterMsg"][1]["row"][i]["msg"]
            , List["DisasterMsg"][1]["row"][i]["send_platform"]
        ] \
        for i in range(len(List["DisasterMsg"][1]["row"]))
    ]

In [5]:
# parse 합치기
def concat_parse(url, serviceKey, pageNo, numOfRows, _type) -> str :
    return url + \
     "?serviceKey=" + serviceKey + \
     "&pageNo=" + pageNo + \
     "&numOfRows=" + numOfRows + \
     "&type=" + _type

In [6]:
# parse check
tot = concat_parse(url, serviceKey, pageNo, numOfRows, _type)
#print(tot)

In [7]:
error_code = [290, 310, 333, 336, 337, 500, 600, 601, 300]
res = []
idx = 1

try :        
    while True :        
        print("idx : ", idx)
        
        pageNo = str(idx)
        tot = concat_parse(url, serviceKey, pageNo, numOfRows, _type)
        response = urlopen(tot)
        
        getCode = response.getcode()
        if getCode in error_code :
            print("getcode() : ", getCode, " => break!")
            break
            
        response_body = response.read().decode('utf-8')
        json_object = json.loads(response_body)
        tmp = return_list_from_respose(json_object)
        res.extend(tmp)    
        
        print(idx," 완료")
        idx += 1
        print("================")
        
except Exception as ex:
    # 여기서 진짜 에러와 끝이란 걸 어떻게 알 수 있지?
    # 일단, 끝이 있으니까 그만 돌게 한 의도는 설계 완료!
    print("error : ", ex)

idx :  1
1  완료
idx :  2
2  완료
idx :  3
3  완료
idx :  4
4  완료
idx :  5
5  완료
idx :  6
error :  'DisasterMsg'


In [8]:
# 테이블 변환
parsing_table = pd.DataFrame(res, columns = coloumns)
parsing_table.head()

,create_date,location_id,location_name,md101_sn,msg,send_platform
0,2020/02/20 10:05:18,217,제주특별자치도 전체,25440,"[제주특별자치도] 제주에는 코로나19 환자가 없지만 여전히 ""경계"" 상태를 유지하고...",cbs
1,2020/02/20 09:07:00,93,경상북도 의성군,25439,"[의성군재난안전대책본부]발열,기침 등 호흡기 증상시 보건소(830-6662) 또는 ...",cbs
2,2020/02/20 09:01:33,83,경상북도 상주시,25438,"[상주시청]계림동 여성1명 코로나19 양성 판정, 확진 발표 예정.\r\n마스크착용...",cbs
3,2020/02/19 20:55:22,53,경상남도 전체,25437,[경상남도청]대구경북지역 코로나19 확진자 발생. \n지역전파 차단을 위해 예방수칙...,cbs
4,2020/02/19 20:43:24,122,부산광역시 기장군,25436,[기장군청] 대구경북지역 코로나19 확진자 19명발생. 지역전파차단을 위해 예방수칙...,cbs


In [9]:
parsing_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4935 entries, 0 to 4934
Data columns (total 6 columns):
create_date      4935 non-null object
location_id      4935 non-null object
location_name    4935 non-null object
md101_sn         4935 non-null object
msg              4935 non-null object
send_platform    4935 non-null object
dtypes: object(6)
memory usage: 231.4+ KB


In [10]:
parsing_table.to_csv('disaster_msg.csv'
                     , index = False
                     , encoding = 'cp949')